In [1]:
import os
import json
import time
import pandas as pd
import numpy as np
from pathlib import Path
from openai import OpenAI
import re
import torch
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

os.environ["DASHSCOPE_API_KEY"] = "sk-2ea9416b45e04af6b6aa72d3c2ade52f"

# 设置环境变量和配置
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 避免警告
OUTPUT_DIR = "生成结果/last_quadrant"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 初始化OpenAI客户端
client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

import os
import glob

data_files = {
    "excel_file_1": "生成结果/last_quadrant/四象限数据-痛点.xlsx",
    "excel_file_2": "生成结果/last_quadrant/四象限数据-需求.xlsx",
    "excel_file_3": "生成结果/last_quadrant/四象限数据-场景匹配.xlsx",
    "txt_file_4": "生成结果/social_media/Comment-归一化的模型结果.txt",
    "txt_file_5": "生成结果/social_media/Like-归一化的模型结果.txt",
    "txt_file_6": "生成结果/social_media/Repost-归一化的模型结果.txt",
    "txt_file_7": "生成结果/social_media/Share-归一化的模型结果.txt",
    "txt_file_8": "生成结果/social_media/View-归一化的模型结果.txt",
    "md_file_1": "生成结果/social_media/理论假设.md"
}

# 自动搜索三个文件夹中的txt文件
defect_dir = "生成结果/defect_analysis/"
needs_dir = "生成结果/needs_analysis/"
matches_dir = "生成结果/matches_analysis/"

# 在每个文件夹中查找第一个txt文件
def find_first_txt(directory):
    txt_files = glob.glob(os.path.join(directory, "*.txt"))
    return txt_files[0] if txt_files else None

# 更新data_files字典
defect_txt = find_first_txt(defect_dir)
if defect_txt:
    data_files["txt_file_1"] = defect_txt
else:
    print(f"警告: 在 {defect_dir} 中未找到txt文件")

needs_txt = find_first_txt(needs_dir)
if needs_txt:
    data_files["txt_file_2"] = needs_txt
else:
    print(f"警告: 在 {needs_dir} 中未找到txt文件")

matches_txt = find_first_txt(matches_dir)
if matches_txt:
    data_files["txt_file_3"] = matches_txt
else:
    print(f"警告: 在 {matches_dir} 中未找到txt文件")

# 打印更新后的文件路径
for key in ["txt_file_1", "txt_file_2", "txt_file_3"]:
    if key in data_files:
        print(f"{key}: {data_files[key]}")

# Prompt文件路径列表
prompt_files = {
    "prompt_1": "Data/建议表1-表达-流量关系表Prompt.txt",
    "prompt_2": "Data/建议表2-消费者旅程不同阶段的产品表达关注表Prompt.txt",
    "prompt_3": "Data/建议表3-品类功能利益点情感利益点分析表Prompt.txt",
    "prompt_4": "Data/建议表4-产品改进建议Prompt.txt",
    "prompt_5": "Data/建议表5-情感利益点改进建议Prompt.txt",
    "prompt_6": "Data/建议表6-改进优先级排序Prompt.txt",
}

txt_file_1: 生成结果/defect_analysis\致欧-2025-01-10之后VOC数据_缺陷象限分析_完整列表.txt
txt_file_2: 生成结果/needs_analysis\致欧-2025-01-10之后VOC数据_用户需求象限分析_完整列表.txt
txt_file_3: 生成结果/matches_analysis\致欧-2025-01-10之后VOC数据_功能场景匹配象限分析_完整列表.txt


In [2]:
from sentence_transformers import SentenceTransformer, util

# 为RTX 4090D选择高性能中文模型
# 对于中文+数字的组合文本，我们选择性能更高的中文检索模型
MODEL_NAME = "BAAI/bge-large-zh-v1.5"  # 高性能中文模型，约1.3GB

def load_model(model_name=MODEL_NAME):
    """加载或下载嵌入模型"""
    print(f"正在加载嵌入模型 {model_name}...")
    try:
        # 设置设备，使用GPU加速
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cuda":
            print(f"将使用GPU: {torch.cuda.get_device_name(0)}")
        else:
            print("未检测到GPU，将使用CPU")
        
        # 加载模型
        model = SentenceTransformer(model_name, device=device)
        
        # 验证模型
        test_embedding = model.encode("测试句子", convert_to_tensor=True)
        print(f"模型加载成功! 嵌入维度: {test_embedding.shape[0]}")
        return model
    except Exception as e:
        print(f"模型加载失败: {e}")
        print("请确保网络连接正常，或尝试使用其他模型")
        return None

# 加载模型
embedding_model = load_model()

正在加载嵌入模型 BAAI/bge-large-zh-v1.5...
将使用GPU: NVIDIA GeForce RTX 4090 D


模型加载成功! 嵌入维度: 1024


In [3]:
# 用于估算token数量的简单函数
def estimate_tokens(text):
    # 一个简单的估算：按空格分词，每个词约1.3个token（中文需要调整）
    chinese_char_count = sum(1 for c in text if '\u4e00' <= c <= '\u9fff')
    english_word_count = len(text.split()) - chinese_char_count / 5  # 粗略调整
    return int(chinese_char_count * 1.5 + english_word_count * 1.3)

In [4]:
# 文档处理类
class DocumentProcessor:
    def __init__(self, data_files):
        self.data_files = data_files
        self.chunks = []
        self.file_stats = {}
        
    def process_all_files(self):
        """处理所有文件，创建分块"""
        print("开始处理数据文件...")
        
        # 处理Excel文件
        for key in [k for k in self.data_files.keys() if k.startswith("excel")]:
            self._process_excel_file(key, self.data_files[key])
        
        # 处理文本和MD文件
        for key in [k for k in self.data_files.keys() if k.startswith("txt") or k.startswith("md")]:
            self._process_text_file(key, self.data_files[key])
            
        print(f"文件处理完成，共创建 {len(self.chunks)} 个文档块")
        return self.chunks, self.file_stats
    
    def _process_excel_file(self, file_key, file_path):
        """处理Excel文件，创建有意义的文档块"""
        try:
            df = pd.read_excel(file_path)
            
            # 存储文件统计信息
            self.file_stats[file_key] = {
                "rows": len(df),
                "columns": len(df.columns),
                "column_names": df.columns.tolist()
            }
            
            # 策略1：完整表格（如果不太大）
            full_content = f"文件: {file_key} (完整表格)\n表格概览: {len(df)}行 x {len(df.columns)}列\n"
            full_content += f"列名: {', '.join(str(col) for col in df.columns)}\n\n"
            full_content += df.to_string(index=False)
            
            full_tokens = estimate_tokens(full_content)
            
            # 如果完整表格不太大，直接作为一个块
            if full_tokens < 15000:
                self.chunks.append({
                    "id": f"{file_key}_full",
                    "content": full_content,
                    "source": file_key,
                    "chunk_type": "excel_full",
                    "tokens": full_tokens
                })
                print(f"添加完整表格: {file_key} ({full_tokens} tokens)")
                return
            
            # 策略2：表格元数据 + 列名说明
            metadata = f"文件: {file_key} (表格元数据)\n"
            metadata += f"表格概览: {len(df)}行 x {len(df.columns)}列\n"
            metadata += f"列名及说明:\n"
            
            # 尝试推断每列的数据类型和基本统计信息
            for col in df.columns:
                col_type = str(df[col].dtype)
                metadata += f"- {col} (类型: {col_type})"
                
                # 对数值列添加统计信息
                if pd.api.types.is_numeric_dtype(df[col]):
                    metadata += f": 范围 {df[col].min()}-{df[col].max()}, 平均值 {df[col].mean():.2f}, 中位数 {df[col].median()}"
                # 对分类列添加唯一值信息
                elif pd.api.types.is_categorical_dtype(df[col]) or df[col].nunique() < 10:
                    unique_vals = ", ".join(str(v) for v in df[col].unique()[:5])
                    metadata += f": 值包括 {unique_vals}"
                    if df[col].nunique() > 5:
                        metadata += f" 等共{df[col].nunique()}个唯一值"
                # 对日期列添加时间范围
                elif pd.api.types.is_datetime64_dtype(df[col]):
                    metadata += f": 时间范围 {df[col].min()} 到 {df[col].max()}"
                # 对文本列添加长度信息
                elif pd.api.types.is_string_dtype(df[col]) or df[col].dtype == object:
                    try:
                        if df[col].notna().any():
                            avg_len = df[col].astype(str).str.len().mean()
                            metadata += f": 平均长度 {avg_len:.1f}字符"
                    except:
                        pass
                
                metadata += "\n"
            
            self.chunks.append({
                "id": f"{file_key}_metadata",
                "content": metadata,
                "source": file_key,
                "chunk_type": "excel_metadata",
                "tokens": estimate_tokens(metadata)
            })
            
            # 策略3：按列分组
            # 简单方法：相邻的2-3列组合在一起
            col_groups = []
            current_group = []
            for col in df.columns:
                current_group.append(col)
                if len(current_group) >= 3:  # 每组最多3列
                    col_groups.append(current_group)
                    current_group = []
            
            # 添加剩余的列
            if current_group:
                col_groups.append(current_group)
            
            # 为每个列组创建内容块
            for i, group in enumerate(col_groups):
                # 准备列组内容
                group_content = f"文件: {file_key} (列组 {i+1})\n"
                group_content += f"包含列: {', '.join(str(col) for col in group)}\n\n"
                
                # 添加这些列的数据
                group_df = df[group]
                group_content += group_df.to_string(index=True)
                
                self.chunks.append({
                    "id": f"{file_key}_colgroup_{i+1}",
                    "content": group_content,
                    "source": file_key,
                    "chunk_type": "excel_column_group",
                    "columns": group,
                    "tokens": estimate_tokens(group_content)
                })
            
            # 策略4：按行分块，但保留列名
            rows_per_chunk = 50  # 根据表格复杂度调整
            
            for i in range(0, len(df), rows_per_chunk):
                end_idx = min(i + rows_per_chunk, len(df))
                chunk_df = df.iloc[i:end_idx]
                
                rows_content = f"文件: {file_key} (行 {i+1}-{end_idx}，共{len(df)}行)\n"
                rows_content += f"列名: {', '.join(str(col) for col in df.columns)}\n\n"
                rows_content += chunk_df.to_string(index=True)
                
                self.chunks.append({
                    "id": f"{file_key}_rows_{i+1}_{end_idx}",
                    "content": rows_content,
                    "source": file_key,
                    "chunk_type": "excel_rows",
                    "row_range": [i+1, end_idx],
                    "tokens": estimate_tokens(rows_content)
                })
            
            print(f"处理完成: {file_key}, 创建了1个元数据块、{len(col_groups)}个列组块和{(len(df)-1)//rows_per_chunk+1}个行块")
            
        except Exception as e:
            print(f"处理 {file_key} 时出错: {e}")
            # 添加错误信息
            self.chunks.append({
                "id": f"{file_key}_error",
                "content": f"文件: {file_key}\n处理错误: {e}",
                "source": file_key,
                "chunk_type": "error",
                "tokens": 50
            })
    
    def _process_text_file(self, file_key, file_path):
        """处理文本文件，创建有意义的文档块"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            
            # 存储文件统计信息
            line_count = content.count('\n') + 1
            self.file_stats[file_key] = {
                "characters": len(content),
                "lines": line_count
            }
            
            # 文件较小时，保留为一个完整块
            if len(content) < 12000:
                full_content = f"文件: {file_key} (完整内容)\n\n{content}"
                
                self.chunks.append({
                    "id": f"{file_key}_full",
                    "content": full_content,
                    "source": file_key,
                    "chunk_type": "text_full",
                    "tokens": estimate_tokens(full_content)
                })
                
                print(f"添加完整文本: {file_key} ({len(content)} 字符)")
                return
            
            # 添加文件摘要/概览
            summary = f"文件: {file_key} (概览)\n"
            # 避免在f-string中使用转义字符，先计算行数再使用变量
            summary += f"字符数: {len(content)}, 行数: {line_count}\n"
            
            # 提取文件前几行作为预览
            first_lines = "\n".join(content.split('\n')[:10])
            summary += f"文件开头:\n{first_lines}\n..."
            
            self.chunks.append({
                "id": f"{file_key}_summary",
                "content": summary,
                "source": file_key,
                "chunk_type": "text_summary",
                "tokens": estimate_tokens(summary)
            })
            
            # 检测文档结构
            has_headers = bool(re.search(r'^#+\s+', content, re.MULTILINE))
            has_sections = bool(re.search(r'^[0-9]+\.\s+', content, re.MULTILINE))
            
            # 根据文档结构选择分块策略
            if has_headers and file_key.startswith("md"):
                # 按Markdown标题分块
                self._split_by_markdown_headers(file_key, content)
            elif has_sections:
                # 按编号章节分块
                self._split_by_numbered_sections(file_key, content)
            else:
                # 按语义段落分块
                self._split_by_semantic_paragraphs(file_key, content)
            
        except Exception as e:
            print(f"处理 {file_key} 时出错: {e}")
            self.chunks.append({
                "id": f"{file_key}_error",
                "content": f"文件: {file_key}\n处理错误: {e}",
                "source": file_key,
                "chunk_type": "error",
                "tokens": 50
            })
    
    def _split_by_markdown_headers(self, file_key, content):
        """按Markdown标题分块，保留标题层次结构"""
        # 识别所有标题行及其位置
        header_pattern = re.compile(r'^(#+)\s+(.*?)$', re.MULTILINE)
        headers = [(m.group(1), m.group(2), m.start()) for m in header_pattern.finditer(content)]
        
        if not headers:
            # 没有找到标题，回退到段落分块
            self._split_by_semantic_paragraphs(file_key, content)
            return
        
        # 添加文件结尾位置
        headers.append((None, None, len(content)))
        
        # 按标题分块
        for i in range(len(headers) - 1):
            level, title, start = headers[i]
            _, _, end = headers[i + 1]
            
            section_content = content[start:end]
            
            # 创建块内容，包含上下文信息
            if i > 0:
                # 添加父标题作为上下文
                parent_idx = i - 1
                while parent_idx >= 0:
                    parent_level, parent_title, _ = headers[parent_idx]
                    if parent_level and len(parent_level) < len(level):
                        header_context = f"文件: {file_key}\n章节: {parent_title} > {title}\n\n"
                        break
                    parent_idx -= 1
                else:
                    header_context = f"文件: {file_key}\n章节: {title}\n\n"
            else:
                header_context = f"文件: {file_key}\n章节: {title}\n\n"
            
            chunk_content = header_context + section_content
            chunk_tokens = estimate_tokens(chunk_content)
            
            # 如果块太大，进一步分割
            if chunk_tokens > 4000:
                # 按段落分割大块
                paragraphs = re.split(r'\n\s*\n', section_content)
                
                # 创建包含标题的首块
                first_para = header_context + paragraphs[0]
                self.chunks.append({
                    "id": f"{file_key}_h{len(level)}_{i}_0",
                    "content": first_para,
                    "source": file_key,
                    "chunk_type": "markdown_section_start",
                    "header": title,
                    "level": len(level),
                    "tokens": estimate_tokens(first_para)
                })
                
                # 分割剩余段落，保持上下文
                current_chunk = ""
                chunk_num = 1
                
                for j, para in enumerate(paragraphs[1:], 1):
                    # 为每个新段添加上下文信息
                    para_with_context = f"文件: {file_key}\n章节: {title} (续)\n\n{para}"
                    para_tokens = estimate_tokens(para_with_context)
                    
                    # 如果添加这个段落会使块太大，创建新块
                    if estimate_tokens(current_chunk + para_with_context) > 3000:
                        if current_chunk:
                            self.chunks.append({
                                "id": f"{file_key}_h{len(level)}_{i}_{chunk_num}",
                                "content": current_chunk,
                                "source": file_key,
                                "chunk_type": "markdown_section_continued",
                                "header": title,
                                "level": len(level),
                                "tokens": estimate_tokens(current_chunk)
                            })
                            chunk_num += 1
                            current_chunk = para_with_context
                    else:
                        current_chunk += "\n\n" + para_with_context if current_chunk else para_with_context
                
                # 添加最后一个块
                if current_chunk:
                    self.chunks.append({
                        "id": f"{file_key}_h{len(level)}_{i}_{chunk_num}",
                        "content": current_chunk,
                        "source": file_key,
                        "chunk_type": "markdown_section_end",
                        "header": title,
                        "level": len(level),
                        "tokens": estimate_tokens(current_chunk)
                    })
            else:
                # 块大小合适，直接添加
                self.chunks.append({
                    "id": f"{file_key}_h{len(level)}_{i}",
                    "content": chunk_content,
                    "source": file_key,
                    "chunk_type": "markdown_section",
                    "header": title,
                    "level": len(level),
                    "tokens": chunk_tokens
                })
        
        print(f"按Markdown标题分块: {file_key}, 创建了{len(self.chunks) - 1}个块")
    
    def _split_by_numbered_sections(self, file_key, content):
        """按编号章节分块"""
        # 识别所有编号章节标题
        section_pattern = re.compile(r'^(\d+\.\s*.*?)$', re.MULTILINE)
        sections = [(m.group(1), m.start()) for m in section_pattern.finditer(content)]
        
        if not sections:
            # 没有找到章节，回退到段落分块
            self._split_by_semantic_paragraphs(file_key, content)
            return
        
        # 添加文件结尾位置
        sections.append(("END", len(content)))
        
        # 按章节分块
        for i in range(len(sections) - 1):
            title, start = sections[i]
            _, end = sections[i + 1]
            
            section_content = content[start:end]
            
            # 创建块内容
            chunk_content = f"文件: {file_key}\n章节: {title}\n\n{section_content}"
            chunk_tokens = estimate_tokens(chunk_content)
            
            # 如果块太大，按段落进一步拆分
            if chunk_tokens > 4000:
                paragraphs = re.split(r'\n\s*\n', section_content)
                
                # 分批处理段落
                current_chunk = f"文件: {file_key}\n章节: {title}\n\n{paragraphs[0]}"
                current_tokens = estimate_tokens(current_chunk)
                chunk_num = 0
                
                for para in paragraphs[1:]:
                    para_tokens = estimate_tokens(para)
                    
                    # 如果添加这个段落会使块太大，创建新块
                    if current_tokens + para_tokens > 3500:
                        self.chunks.append({
                            "id": f"{file_key}_section_{i}_{chunk_num}",
                            "content": current_chunk,
                            "source": file_key,
                            "chunk_type": "numbered_section_part",
                            "section": title,
                            "part": chunk_num,
                            "tokens": current_tokens
                        })
                        chunk_num += 1
                        current_chunk = f"文件: {file_key}\n章节: {title} (续)\n\n{para}"
                        current_tokens = estimate_tokens(current_chunk)
                    else:
                        current_chunk += f"\n\n{para}"
                        current_tokens += para_tokens
                
                # 添加最后一个块
                if current_chunk:
                    self.chunks.append({
                        "id": f"{file_key}_section_{i}_{chunk_num}",
                        "content": current_chunk,
                        "source": file_key,
                        "chunk_type": "numbered_section_part",
                        "section": title,
                        "part": chunk_num,
                        "tokens": current_tokens
                    })
            else:
                # 块大小合适，直接添加
                self.chunks.append({
                    "id": f"{file_key}_section_{i}",
                    "content": chunk_content,
                    "source": file_key,
                    "chunk_type": "numbered_section",
                    "section": title,
                    "tokens": chunk_tokens
                })
        
        print(f"按编号章节分块: {file_key}, 创建了{len(sections) - 1}个章节块")
    
    def _split_by_semantic_paragraphs(self, file_key, content):
        """按语义段落分块，保留上下文连贯性"""
        # 分割成自然段落
        paragraphs = re.split(r'\n\s*\n', content)
        
        # 使用滑动窗口方法，保持上下文重叠
        window_size = 5  # 每个窗口包含的段落数
        stride = 3       # 窗口滑动步长
        
        for i in range(0, len(paragraphs), stride):
            # 获取当前窗口的段落
            window_paras = paragraphs[i:i + window_size]
            
            if not window_paras:
                continue
                
            # 创建块内容
            start_idx = i + 1
            end_idx = min(i + len(window_paras), len(paragraphs))
            
            chunk_content = f"文件: {file_key} (段落 {start_idx}-{end_idx}，共{len(paragraphs)}段)\n\n"
            chunk_content += "\n\n".join(window_paras)
            
            self.chunks.append({
                "id": f"{file_key}_para_{start_idx}_{end_idx}",
                "content": chunk_content,
                "source": file_key,
                "chunk_type": "semantic_paragraphs",
                "paragraph_range": [start_idx, end_idx],
                "tokens": estimate_tokens(chunk_content)
            })
        
        print(f"按语义段落分块: {file_key}, 创建了{(len(paragraphs) - 1) // stride + 1}个块")

In [5]:
# 嵌入模型检索系统
class EmbeddingRetriever:
    def __init__(self, chunks, model):
        self.chunks = chunks
        self.model = model
        
        # 初始化文件索引
        self.file_index = {}
        for chunk in chunks:
            source = chunk["source"]
            if source not in self.file_index:
                self.file_index[source] = []
            self.file_index[source].append(chunk)
        
        # 构建索引
        self._build_index()
    
    # def _build_index(self):
    #     """构建嵌入索引"""
    #     print("构建嵌入索引...")
        
    #     # 准备文档内容
    #     self.chunk_contents = [chunk["content"] for chunk in self.chunks]
        
    #     # 计算嵌入 - 分批处理以避免内存溢出
    #     batch_size = 32
    #     all_embeddings = []
        
    #     # 使用tqdm展示进度
    #     for i in tqdm(range(0, len(self.chunk_contents), batch_size), desc="生成嵌入向量"):
    #         batch = self.chunk_contents[i:i+batch_size]
    #         batch_embeddings = self.model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
    #         all_embeddings.append(batch_embeddings)
        
    #     # 合并所有批次的嵌入
    #     self.embeddings = torch.cat(all_embeddings)
        
    #     print(f"索引构建完成，文档数: {len(self.chunks)}, 嵌入维度: {self.embeddings.shape[1]}")
    def _build_index(self):
        """构建嵌入索引"""
        print("构建嵌入索引...")
        
        # 准备文档内容 - 这行没有问题，但应该在使用前定义
        self.chunk_contents = [chunk["content"] for chunk in self.chunks]
        
        # 为RTX 4090D优化的批处理设置
        batch_size = 64  # 可以使用更大的批次
        
        # 如果使用CUDA，设置自动混合精度计算来加速
        if torch.cuda.is_available():
            print("启用自动混合精度计算以提高性能")
            with torch.cuda.amp.autocast():
                all_embeddings = []
                for i in tqdm(range(0, len(self.chunk_contents), batch_size), desc="生成嵌入向量"):
                    batch = self.chunk_contents[i:i+batch_size]
                    batch_embeddings = self.model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
                    all_embeddings.append(batch_embeddings)
        else:
            # CPU处理
            all_embeddings = []
            for i in tqdm(range(0, len(self.chunk_contents), batch_size), desc="生成嵌入向量"):
                batch = self.chunk_contents[i:i+batch_size]
                batch_embeddings = self.model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
                all_embeddings.append(batch_embeddings)
        
        # 合并所有批次的嵌入
        self.embeddings = torch.cat(all_embeddings)
        
        print(f"索引构建完成，文档数: {len(self.chunks)}, 嵌入维度: {self.embeddings.shape[1]}")
    
    def get_file_list(self):
        """获取所有文件名列表"""
        return list(self.file_index.keys())
    
    def retrieve_files(self, query, top_k=5):
        """检索与查询最相关的文件"""
        # 生成查询嵌入
        query_embedding = self.model.encode(query, convert_to_tensor=True)
        
        # 计算每个块的相似度
        chunk_similarities = util.cos_sim(query_embedding, self.embeddings)[0].cpu().numpy()
        
        # 计算每个文件的加权分数
        file_scores = {}
        for i, chunk in enumerate(self.chunks):
            source = chunk["source"]
            if source not in file_scores:
                file_scores[source] = {"total": 0, "count": 0, "max": 0}
            
            file_scores[source]["total"] += chunk_similarities[i]
            file_scores[source]["count"] += 1
            file_scores[source]["max"] = max(file_scores[source]["max"], chunk_similarities[i])
        
        # 使用加权组合：最大相似度有70%权重，平均相似度有30%权重
        for source in file_scores:
            avg_sim = file_scores[source]["total"] / file_scores[source]["count"]
            max_sim = file_scores[source]["max"]
            file_scores[source]["score"] = 0.7 * max_sim + 0.3 * avg_sim
        
        # 排序文件
        sorted_files = sorted(
            [{"id": source, "relevance": float(file_scores[source]["score"])} 
             for source in file_scores],
            key=lambda x: x["relevance"],
            reverse=True
        )
        
        return sorted_files[:top_k]
    
    # 在EmbeddingRetriever类中修改retrieve方法
    def retrieve(self, query, top_k=25, max_tokens=45000, similarity_threshold=0.35):  # 增加参数
        """检索与查询最相关的文档块"""
        # 生成查询嵌入
        query_embedding = self.model.encode(query, convert_to_tensor=True)
        
        # 计算相似度
        similarities = util.cos_sim(query_embedding, self.embeddings)[0].cpu().numpy()
        
        # 应用相似度阈值过滤
        qualified_indices = np.where(similarities > similarity_threshold)[0]
        
        # 如果结果太少，回退到top-k策略
        if len(qualified_indices) < 5:
            print(f"注意: 只有 {len(qualified_indices)} 个文档块超过相似度阈值 {similarity_threshold}，采用前70个索引")
            top_indices = similarities.argsort()[-70:][::-1]  # 扩大候选池
        else:
            # 按相似度从高到低排序
            top_indices = qualified_indices[np.argsort(-similarities[qualified_indices])]
            if len(top_indices) > 70:
                top_indices = top_indices[:70]  # 限制候选池大小
        
        # 按相关性组织结果
        results = []
        current_tokens = estimate_tokens(query)
        sources_included = set()
        
        # 首先确保包含各文件中最相关的块
        for idx in top_indices:
            chunk = self.chunks[idx]
            source = chunk["source"]
            
            # 如果这个源文件还没有被包含，且token数量允许
            if source not in sources_included and current_tokens + chunk["tokens"] <= max_tokens:
                results.append({
                    "chunk": chunk,
                    "similarity": float(similarities[idx])
                })
                current_tokens += chunk["tokens"]
                sources_included.add(source)
                
                # 如果已经包含了所有必要文件，且达到一定数量，可以提前结束
                if len(sources_included) >= len(self.file_index) or len(results) >= 10:
                    break
        
        # 然后添加其他高相关性块，直到达到token限制
        for idx in top_indices:
            chunk = self.chunks[idx]
            
            # 检查是否已经包含在结果中
            if not any(r["chunk"]["id"] == chunk["id"] for r in results):
                # 如果添加这个块不会超过token限制
                if current_tokens + chunk["tokens"] <= max_tokens * 0.9:  # 保留10%的余量
                    results.append({
                        "chunk": chunk,
                        "similarity": float(similarities[idx])
                    })
                    current_tokens += chunk["tokens"]
            
            # 如果已经达到top_k或接近token限制，就停止
            if len(results) >= top_k or current_tokens > max_tokens * 0.85:
                break
        
        # 估算提示token使用量
        estimated_prompt_tokens = 1500  # 估计其他提示内容的token数
        safe_token_limit = max_tokens - estimated_prompt_tokens
        
        # 如果仍然超出安全限制，进行裁剪
        if current_tokens > safe_token_limit:
            print(f"警告: 检索结果超出安全token限制，进行裁剪 ({current_tokens} > {safe_token_limit})")
            
            # 保留必要文件和高相关性块
            sorted_results = sorted(results, key=lambda x: x["similarity"], reverse=True)
            trimmed_results = []
            trimmed_tokens = 0
            sources_kept = set()
            
            # 首先保证每个源文件至少有一个块
            for result in sorted_results:
                source = result["chunk"]["source"]
                if source not in sources_kept:
                    trimmed_results.append(result)
                    trimmed_tokens += result["chunk"]["tokens"]
                    sources_kept.add(source)
                    
                    if trimmed_tokens >= safe_token_limit * 0.7:  # 使用70%的token预算保留每个文件
                        break
            
            # 然后添加高相关性的块直到达到限制
            for result in sorted_results:
                if result not in trimmed_results and trimmed_tokens + result["chunk"]["tokens"] <= safe_token_limit:
                    trimmed_results.append(result)
                    trimmed_tokens += result["chunk"]["tokens"]
            
            results = trimmed_results
            current_tokens = trimmed_tokens
        
        print(f"检索到 {len(results)} 个文档块，总计约 {current_tokens} tokens (R1输入限制: {max_tokens})")
        
        # 按相似度排序结果
        results.sort(key=lambda x: x["similarity"], reverse=True)
        
        return results, current_tokens
    
    def generate_file_metadata(self):
        """生成文件元数据，用于增强器"""
        metadata = {}
        
        for source, chunks in self.file_index.items():
            # 找出这个文件的元数据块
            metadata_chunks = [c for c in chunks if c["chunk_type"] in ["excel_metadata", "text_summary"]]
            
            if metadata_chunks:
                # 使用元数据块的内容
                chunk = metadata_chunks[0]
                content = chunk["content"]
                
                # 提取文件类型
                file_type = "Unknown"
                if source.startswith("excel"):
                    file_type = "Excel表格"
                elif source.startswith("txt"):
                    file_type = "文本文档"
                elif source.startswith("md"):
                    file_type = "Markdown文档"
                
                # 提取描述信息
                description = source
                if "表格概览" in content:
                    description_match = re.search(r"表格概览: .*行", content)
                    if description_match:
                        description = description_match.group(0)
                elif "字符数" in content:
                    description_match = re.search(r"字符数: .*行", content)
                    if description_match:
                        description = description_match.group(0)
                
                # 提取关键列或章节
                key_items = []
                if file_type == "Excel表格" and "列名:" in content:
                    columns_match = re.search(r"列名: (.*)", content)
                    if columns_match:
                        columns = columns_match.group(1).split(", ")
                        key_items = columns[:5]  # 取前5列作为关键列
                else:
                    # 提取文本文件中可能的章节
                    sections = []
                    for c in chunks:
                        if "章节:" in c["content"]:
                            section_match = re.search(r"章节: ([^>\n]*)", c["content"])
                            if section_match:
                                sections.append(section_match.group(1).strip())
                    key_items = list(set(sections))[:5]  # 去重并取前5个
                
                # 构建元数据
                metadata[source] = {
                    "type": file_type,
                    "description": description,
                    "key_items": key_items
                }
            else:
                # 如果没有元数据块，使用基本信息
                file_type = "未知文件类型"
                if source.startswith("excel"):
                    file_type = "Excel表格"
                elif source.startswith("txt"):
                    file_type = "文本文档"
                elif source.startswith("md"):
                    file_type = "Markdown文档"
                    
                metadata[source] = {
                    "type": file_type,
                    "description": source,
                    "key_items": []
                }
        
        return metadata

In [6]:
# 文本增强器 - 使用检索结果生成结构化上下文
class TextRetrieverAugmenter:
    def __init__(self, retriever):
        self.retriever = retriever
        self.file_metadata = retriever.generate_file_metadata()
    
    def augment_query(self, query, prompt_to_r1, max_files=5, max_context_tokens=45000):  # 增加默认token限制
        """增强查询，生成适合R1的详细上下文"""
        # 1. 检索相关文件
        relevant_files = self.retriever.retrieve_files(query, top_k=max_files)
        
        # 2. 生成增强上下文
        augmented_context = self._generate_structured_context(query, relevant_files)
        
        # 3. 估算上下文token数量
        context_tokens = estimate_tokens(augmented_context)
        
        # 4. 如果超出限制，进行缩减
        if context_tokens > max_context_tokens:
            # 简化策略: 保留较少的相关文件
            reduced_files = max(2, int(max_files * max_context_tokens / context_tokens))
            relevant_files = self.retriever.retrieve_files(query, top_k=reduced_files)
            augmented_context = self._generate_structured_context(query, relevant_files, simplified=True)
        
        # 5. 融合到R1提示
        enhanced_prompt = f"""
    {prompt_to_r1}
    
    <RETRIEVAL_RESULTS>
    {augmented_context}
    </RETRIEVAL_RESULTS>
    
    请基于以上检索结果中提供的文件内容进行分析，按照任务要求生成响应。
    请特别关注"PRIMARY_FILES"部分的关键信息，并参考"RETRIEVAL_GUIDANCE"提供的分析策略。
    请根据Prompt生成结果后，进行检查，确保只输出最好、最完整的一个结果。
    """
        return enhanced_prompt, relevant_files
    
    def _generate_structured_context(self, query, relevant_files, simplified=False):
        """生成结构化上下文信息"""
        # 将相关文件分为主要和次要文件
        primary_files = relevant_files[:min(3, len(relevant_files))]
        secondary_files = relevant_files[3:min(5, len(relevant_files))]
        
        output = "<FILE_SELECTION>\n"
        output += "以下是与查询相关的文件列表，按相关性从高到低排序：\n\n"
        
        # 添加主要文件
        output += "<PRIMARY_FILES>\n"
        for i, file in enumerate(primary_files):
            file_id = file["id"]
            relevance = file["relevance"]
            metadata = self.file_metadata.get(file_id, {})
            
            output += f"{i+1}. {file_id} [相关性:{relevance:.2f}]\n"
            output += f"   - 文件类型: {metadata.get('type', '未知类型')}\n"
            output += f"   - 主要内容: {metadata.get('description', '未知内容')}\n"
            
            # 添加关键项目
            key_items = metadata.get('key_items', [])
            if key_items:
                item_type = "关键列" if metadata.get('type', '').startswith("Excel") else "关键章节"
                output += f"   - {item_type}: {', '.join(key_items)}\n"
            
            output += "\n"
        
        # 添加次要文件
        if not simplified and secondary_files:
            output += "<SECONDARY_FILES>\n"
            for i, file in enumerate(secondary_files):
                file_id = file["id"]
                relevance = file["relevance"]
                metadata = self.file_metadata.get(file_id, {})
                
                output += f"{i+len(primary_files)+1}. {file_id} [相关性:{relevance:.2f}]\n"
                output += f"   - 文件类型: {metadata.get('type', '未知类型')}\n"
                output += f"   - 主要内容: {metadata.get('description', '未知内容')}\n"
                output += f"   - 参考价值: 辅助分析信息\n\n"
        
        # 添加文件间关系
        output += "<DATA_RELATIONSHIPS>\n"
        
        # 识别特定文件间的关系
        excel_files = [f["id"] for f in relevant_files if f["id"].startswith("excel")]
        txt_files = [f["id"] for f in relevant_files if f["id"].startswith("txt")]
        md_files = [f["id"] for f in relevant_files if f["id"].startswith("md")]
        
        # 分析文件关系
        if len(primary_files) >= 2:
            file1 = primary_files[0]["id"]
            file2 = primary_files[1]["id"]
            
            # 检查是否是同一类型的文件
            if file1.startswith("excel") and file2.startswith("excel"):
                output += f"- {file1}与{file2}都是Excel表格，可能包含相关但不同维度的数据，建议交叉分析\n"
            elif file1.startswith("txt") and file2.startswith("txt"):
                output += f"- {file1}与{file2}都是文本文档，可能分别包含不同角度的分析，应结合阅读\n"
            else:
                output += f"- {file1}与{file2}是不同类型的文件，可能一个包含数据而另一个包含分析结果\n"
        
        if excel_files and txt_files:
            output += f"- Excel文件({', '.join(excel_files[:2])})提供了结构化数据，而文本文件({', '.join(txt_files[:2])})可能包含对这些数据的分析和解释\n"
        
        if txt_files and md_files:
            output += f"- 文本文件({', '.join(txt_files[:2])})与Markdown文件({', '.join(md_files[:2])})可能包含相关内容，Markdown文件通常提供更结构化的总结\n"
        
        if not excel_files and not txt_files and not md_files:
            output += "- 未检测到明确的文件间关系，请根据内容自行判断\n"
            
        output += "</DATA_RELATIONSHIPS>\n"
        
        # 如果不是简化版本，添加检索指导
        if not simplified:
            output += "\n<RETRIEVAL_GUIDANCE>\n"
            output += "为高效分析这些数据，建议采取以下策略:\n\n"
            
            # 根据文件类型提供不同的建议
            guidance_added = False
            
            if excel_files:
                output += f"1. 首先分析{excel_files[0]}获取基础数据结构和关键指标\n"
                guidance_added = True
                
            if txt_files:
                prefix = "1" if not guidance_added else "2"
                output += f"{prefix}. 结合{txt_files[0]}中的分析文本理解数据含义和背景\n"
                guidance_added = True
                
            if md_files:
                prefix = "1" if not guidance_added else ("3" if excel_files and txt_files else "2")
                output += f"{prefix}. 参考{md_files[0]}中的结论和建议验证您的分析方向\n"
                
            if not guidance_added:
                output += "1. 仔细阅读检索到的所有文件，寻找与查询相关的关键信息\n"
                output += "2. 对比不同文件中的内容，寻找共同点和差异点\n"
            
            output += "\n分析要点:\n"
            output += "- 注意文件间的数据一致性和互补关系\n"
            output += "- 优先关注相关性分数高的文件中的信息\n"
            output += "- 针对需求生成高质量的分析结果，确保表格格式规范\n"
            output += "</RETRIEVAL_GUIDANCE>\n"
        
        return output

In [7]:
# 解析表格函数
def parse_table_to_json(table_text):
    """尝试解析Markdown表格为JSON格式"""
    try:
        # 查找表格
        lines = table_text.strip().split('\n')
        table_start = None
        table_end = None
        
        for i, line in enumerate(lines):
            if '|' in line:
                if table_start is None:
                    table_start = i
                table_end = i
        
        if table_start is None or table_end is None:
            return None
        
        # 提取表格行
        table_lines = lines[table_start:table_end+1]
        
        # 解析表头
        headers = [h.strip() for h in table_lines[0].split('|') if h.strip()]
        
        # 跳过分隔行
        data_start = 2 if len(table_lines) > 1 and ('---' in table_lines[1] or ':--' in table_lines[1]) else 1
        
        # 解析数据行
        results = []
        for i in range(data_start, len(table_lines)):
            line = table_lines[i]
            cells = [c.strip() for c in line.split('|') if c.strip() or c == ' ']
            
            if len(cells) >= len(headers):
                row_data = {}
                for j, header in enumerate(headers):
                    if j < len(cells):
                        row_data[header] = cells[j]
                results.append(row_data)
        
        return results
    except Exception as e:
        print(f"解析表格失败: {e}")
        return None

In [8]:
# 顺序处理多个Prompt的类
class SequentialPromptProcessor:
    def __init__(self, prompt_files, data_files, output_dir, retriever_augmenter):
        self.prompt_files = prompt_files
        self.data_files = data_files
        self.output_dir = Path(output_dir)
        self.retriever_augmenter = retriever_augmenter
        
        # 确保输出目录存在
        os.makedirs(self.output_dir, exist_ok=True)
        
        # 初始化结果存储
        self.results = {}
    
    def process_all_prompts(self):
        """逐个处理所有Prompt文件"""
        # 获取所有Prompt文件
        prompt_items = sorted(self.prompt_files.items())
        total_files = len(prompt_items)
        
        print(f"找到{total_files}个Prompt文件，开始顺序处理...")
        
        # 逐个处理
        for i, (prompt_key, prompt_path) in enumerate(prompt_items, 1):
            print(f"\n[{i}/{total_files}] 处理 {prompt_key}...")
            
            # 处理单个Prompt
            result = self.process_single_prompt(prompt_key, prompt_path)
            
            # 保存结果
            self.results[prompt_key] = result
            
            # 单独保存当前表格结果
            self._save_single_result(prompt_key, result)
            
            # 间隔一下，避免API限制
            if i < total_files:
                print(f"等待3秒后处理下一个Prompt...")
                time.sleep(3)
        
        # 保存所有结果的汇总
        self._save_all_results()
        
        print(f"\n所有Prompt处理完成！结果保存在 {self.output_dir}")
        return self.results
    
    def process_single_prompt(self, prompt_key, prompt_path):
        """处理单个Prompt文件"""
        start_time = time.time()
        
        # 读取Prompt内容
        with open(prompt_path, 'r', encoding='utf-8') as f:
            prompt_text = f.read()
        
        print(f"开始处理Prompt: {prompt_key} (长度: {len(prompt_text)}字符)")
        
        # 提取查询核心 - 用于检索相关文件
        # 使用Prompt的前300个字符作为检索查询
        query = prompt_text[:min(300, len(prompt_text))]
        
        # 使用增强器生成增强上下文
        try:
            enhanced_prompt, relevant_files = self.retriever_augmenter.augment_query(query, prompt_text)
            
            # 输出检索到的相关文件
            file_info = ", ".join([f"{f['id']} ({f['relevance']:.2f})" for f in relevant_files[:3]])
            print(f"检索到主要相关文件: {file_info}")
            
            # 构建最终提示 - 添加额外指导
            final_prompt = f"""
{enhanced_prompt}

请注意:
1. <PRIMARY_FILES>标签内包含与任务最相关的核心文件信息，应作为主要分析依据
2. <SECONDARY_FILES>标签内包含辅助信息，可用于补充分析
3. <DATA_RELATIONSHIPS>标签解释了文件间的关系，有助于综合分析
4. <RETRIEVAL_GUIDANCE>提供了高效分析这些数据的策略建议

在分析过程中，请优先使用相关性更高的信息。
请生成一个格式规范的Markdown表格，确保表格结构完整，列对齐，以满足任务要求。
请确保只生成最完整的一个结果，输出前进行核查。
"""
            # 打印处理情况
            print(f"增强提示生成完成，长度约 {len(final_prompt)} 字符")
            
            # 估算token数量
            estimated_tokens = estimate_tokens(final_prompt)
            print(f"估计输入token数: {estimated_tokens}")
            
            # 检查token数量是否在限制内
            if estimated_tokens > 52000:  # R1模型限制
                print(f"警告: 估计token数超过限制，尝试缩减检索结果...")
                # 简化策略: 保留较少的相关文件
                enhanced_prompt, relevant_files = self.retriever_augmenter.augment_query(
                    query, prompt_text, max_files=5, max_context_tokens=35000)  # 减少文件数量和上下文长度
                
                final_prompt = f"{enhanced_prompt}\n\n请注意优先使用相关性更高的信息，并生成一个格式规范的Markdown表格。"
                estimated_tokens = estimate_tokens(final_prompt)
                print(f"缩减后估计token数: {estimated_tokens}")
            
            # 调用R1模型
            print(f"调用R1模型...")
            completion = client.chat.completions.create(
                model="deepseek-r1",
                messages=[
                    {'role': 'user', 'content': final_prompt}
                ]
            )
            
            result = {
                "reasoning": completion.choices[0].message.reasoning_content,
                "result": completion.choices[0].message.content,
                "retrieved_files": [f["id"] for f in relevant_files],
                "processing_time": time.time() - start_time
            }
            
            print(f"R1响应完成，处理耗时: {result['processing_time']:.2f}秒")
            
            # 尝试解析表格
            table_json = parse_table_to_json(result["result"])
            if table_json:
                print(f"成功解析表格，包含 {len(table_json)} 行数据")
                result["table_data"] = table_json
            
            return result
            
        except Exception as e:
            error_msg = f"处理 {prompt_key} 时出错: {str(e)}"
            print(f"错误: {error_msg}")
            return {
                "error": error_msg,
                "processing_time": time.time() - start_time
            }
    
    def _save_single_result(self, prompt_key, result):
        """保存单个Prompt的结果"""
        # 从prompt_key提取文件名前缀
        # 例如从prompt_1提取出建议表1-表达-流量关系表
        file_prefix = ""
        for pkey, ppath in self.prompt_files.items():
            if pkey == prompt_key:
                # 提取Prompt.txt前面的文字
                match = re.search(r'.*?(?=Prompt\.txt)', os.path.basename(ppath))
                if match:
                    file_prefix = match.group(0).rstrip('-')
                else:
                    file_prefix = os.path.basename(ppath).split('Prompt.txt')[0].rstrip('-')
                break
        
        if not file_prefix:
            file_prefix = prompt_key
        
        # 保存原始结果（包含推理过程等）
        result_path = self.output_dir / f"{file_prefix}_full.json"
        with open(result_path, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
        
        # 单独保存表格结果（Markdown格式）
        if "result" in result:
            table_path = self.output_dir / f"{file_prefix}_table.md"
            with open(table_path, 'w', encoding='utf-8') as f:
                f.write(result["result"])
        
        # 如果成功解析了表格数据，保存Excel格式
        if "table_data" in result and result["table_data"]:
            excel_path = self.output_dir / f"{file_prefix}.xlsx"
            try:
                df = pd.DataFrame(result["table_data"])
                df.to_excel(excel_path, index=False)
                print(f"已保存Excel表格到 {excel_path}")
            except Exception as e:
                print(f"保存Excel文件失败: {e}")
        
        print(f"已保存 {prompt_key} 的结果到 {self.output_dir}")
    
    def _save_all_results(self):
        """保存所有结果的汇总"""
        all_results_path = self.output_dir / "all_tables_results.json"
        with open(all_results_path, 'w', encoding='utf-8') as f:
            json.dump(self.results, f, ensure_ascii=False, indent=2)

In [9]:
# 执行文档处理
processor = DocumentProcessor(data_files)
chunks, file_stats = processor.process_all_files()

开始处理数据文件...


添加完整表格: excel_file_1 (3592 tokens)
添加完整表格: excel_file_2 (3673 tokens)
添加完整表格: excel_file_3 (579 tokens)
添加完整文本: txt_file_4 (3294 字符)
添加完整文本: txt_file_5 (3418 字符)
添加完整文本: txt_file_6 (3306 字符)
添加完整文本: txt_file_7 (3286 字符)
添加完整文本: txt_file_8 (658 字符)
添加完整文本: md_file_1 (5923 字符)
添加完整文本: txt_file_1 (9395 字符)
添加完整文本: txt_file_2 (9582 字符)
添加完整文本: txt_file_3 (1266 字符)
文件处理完成，共创建 12 个文档块


In [10]:
# 创建检索系统 - 使用预训练模型
retriever = EmbeddingRetriever(chunks, embedding_model)

构建嵌入索引...
启用自动混合精度计算以提高性能


生成嵌入向量:   0%|          | 0/1 [00:00<?, ?it/s]

索引构建完成，文档数: 12, 嵌入维度: 1024


In [11]:
# 创建增强器
augmenter = TextRetrieverAugmenter(retriever)

# 创建处理器
processor = SequentialPromptProcessor(prompt_files, data_files, OUTPUT_DIR, augmenter)

In [12]:
# 处理所有Prompt
results = processor.process_all_prompts()

# 输出总结
successful = sum(1 for r in results.values() if "error" not in r)
print(f"\n处理总结:")
print(f"总共处理: {len(results)}个Prompt")
print(f"成功处理: {successful}个Prompt")
print(f"失败处理: {len(results) - successful}个Prompt")

if len(results) - successful > 0:
    print("\n失败的Prompt:")
    for name, result in results.items():
        if "error" in result:
            print(f"- {name}: {result['error']}")

找到6个Prompt文件，开始顺序处理...

[1/6] 处理 prompt_1...
开始处理Prompt: prompt_1 (长度: 2081字符)
检索到主要相关文件: md_file_1 (0.62), excel_file_3 (0.59), excel_file_2 (0.59)
增强提示生成完成，长度约 3517 字符
估计输入token数: 2621
调用R1模型...


R1响应完成，处理耗时: 42.39秒
成功解析表格，包含 3 行数据
已保存Excel表格到 生成结果\last_quadrant\建议表1-表达-流量关系表.xlsx
已保存 prompt_1 的结果到 生成结果\last_quadrant
等待3秒后处理下一个Prompt...



[2/6] 处理 prompt_2...
开始处理Prompt: prompt_2 (长度: 1791字符)
检索到主要相关文件: md_file_1 (0.61), excel_file_3 (0.59), excel_file_2 (0.58)
增强提示生成完成，长度约 3227 字符
估计输入token数: 2286
调用R1模型...


R1响应完成，处理耗时: 42.86秒
成功解析表格，包含 3 行数据
已保存Excel表格到 生成结果\last_quadrant\建议表2-消费者旅程不同阶段的产品表达关注表.xlsx
已保存 prompt_2 的结果到 生成结果\last_quadrant
等待3秒后处理下一个Prompt...



[3/6] 处理 prompt_3...
开始处理Prompt: prompt_3 (长度: 2073字符)
检索到主要相关文件: excel_file_2 (0.59), excel_file_3 (0.56), excel_file_1 (0.56)
增强提示生成完成，长度约 3499 字符
估计输入token数: 2633
调用R1模型...


...

R1响应完成，处理耗时: 42.24秒
成功解析表格，包含 6 行数据
已保存Excel表格到 生成结果\last_quadrant\建议表4-产品改进建议.xlsx
已保存 prompt_4 的结果到 生成结果\last_quadrant
等待3秒后处理下一个Prompt...



[5/6] 处理 prompt_5...
开始处理Prompt: prompt_5 (长度: 2185字符)
检索到主要相关文件: txt_file_2 (0.69), txt_file_3 (0.67), excel_file_3 (0.63)
增强提示生成完成，长度约 3509 字符
估计输入token数: 2683
调用R1模型...


R1响应完成，处理耗时: 40.06秒
成功解析表格，包含 8 行数据
已保存Excel表格到 生成结果\last_quadrant\建议表5-情感利益点改进建议.xlsx
已保存 prompt_5 的结果到 生成结果\last_quadrant
等待3秒后处理下一个Prompt...



[6/6] 处理 prompt_6...
开始处理Prompt: prompt_6 (长度: 2308字符)
检索到主要相关文件: excel_file_3 (0.62), txt_file_2 (0.62), excel_file_1 (0.62)
增强提示生成完成，长度约 3635 字符
估计输入token数: 2841
调用R1模型...


R1响应完成，处理耗时: 43.74秒
成功解析表格，包含 6 行数据
已保存Excel表格到 生成结果\last_quadrant\建议表6-改进优先级排序.xlsx
已保存 prompt_6 的结果到 生成结果\last_quadrant

所有Prompt处理完成！结果保存在 生成结果\last_quadrant

处理总结:
总共处理: 6个Prompt
成功处理: 6个Prompt
失败处理: 0个Prompt


In [13]:
# 查看生成的文件
import glob
generated_files = glob.glob(f"{OUTPUT_DIR}/*.xlsx")
print(f"生成的Excel文件:")
for file in generated_files:
    print(f"- {os.path.basename(file)}")

生成的Excel文件:
- plot_quadrant_data_场景匹配.xlsx
- plot_quadrant_data_痛点.xlsx
- plot_quadrant_data_需求.xlsx
- 四象限数据-场景匹配.xlsx
- 四象限数据-痛点.xlsx
- 四象限数据-需求.xlsx
- 建议表1-表达-流量关系表.xlsx
- 建议表2-消费者旅程不同阶段的产品表达关注表.xlsx
- 建议表3-品类功能利益点情感利益点分析表.xlsx
- 建议表4-产品改进建议.xlsx
- 建议表5-情感利益点改进建议.xlsx
- 建议表6-改进优先级排序.xlsx
